##Import libraries

In [136]:
import graphlab as gl
import pandas as pd

##Load Data

In [137]:
path = '/Users/andremendes/OneDrive/Estudar Projects/JFS/Python Data/'
approved = gl.SFrame.read_csv(path+'train_data_app.csv')
reproved = gl.SFrame.read_csv(path+'train_data_rep.csv')
train_data_text = gl.SFrame.read_csv(path+'train_data_text.csv')
test_data_text = gl.SFrame.read_csv(path+'test_data_text.csv')

PROGRESS: Finished parsing file /Users/andremendes/OneDrive/Estudar Projects/JFS/Python Data/train_data_app.csv
PROGRESS: Parsing completed. Parsed 10 lines in 0.02632 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/andremendes/OneDrive/Estudar Projects/JFS/Python Data/train_data_app.csv
PROGRESS: Parsing completed. Parsed 10 lines in 0.012216 secs.
PROGRESS: Finished parsing file /Users/andremendes/OneDrive/Estudar Projects/JFS/Python Data/train_data_rep.csv
PROGRESS: Parsing completed. Parsed 10 lines in 0.013607 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str]
If parsing fails due to i

##Generate TF-IDF for approved candidates

In [138]:
approved['word_count'] = gl.text_analytics.count_words(approved['all_text'])
tfidf = gl.text_analytics.tf_idf(approved['word_count'])
approved['tfidf'] = tfidf['docs']

##Generate TF-IDF for reproved candidates

In [139]:
reproved['word_count'] = gl.text_analytics.count_words(reproved['all_text'])
tfidf = gl.text_analytics.tf_idf(reproved['word_count'])
reproved['tfidf'] = tfidf['docs']

##Get the email to each candidate as reference

In [140]:
#Train Set
tr = train_data_text.to_dataframe()
candidates_train = list(tr.columns.values)
candidates_train.remove('X1')
cd_train = pd.DataFrame(candidates_train)

#Test Set
tr = test_data_text.to_dataframe()
candidates_test = list(tr.columns.values)
candidates_test.remove('X1')
cd_test = pd.DataFrame(candidates_test)

##Generate TF-IDF to each candidate and compare with TF-IDF for approved and reproved candidates

###Train Set

In [142]:
for email in candidates_train:
    #get the tf-idf for the candidate
    train_data_text[email+'_word_count'] = gl.text_analytics.count_words(train_data_text[email])
    tfidf = gl.text_analytics.tf_idf(train_data_text[email+'_word_count'])
    train_data_text[email+'_tfidf'] = tfidf['docs']
    #initialize the positive and negative column
    train_data_text[email+'_positive'] = train_data_text[email].apply(lambda x: 0.8000000)
    train_data_text[email+'_negative'] = train_data_text[email].apply(lambda x: 0.8000000)
    #prepare a SFrame to be converted to pandas
    sf = gl.SFrame({"positive":train_data_text[email+'_positive'],"negative":train_data_text[email+'_negative']})
    #convert SFrame to pandas because columns in SFrame are immutable, no access to rows
    sf_pd = sf.to_dataframe()
    #Calculate the distance from tf-idf candidate to approved and reproved candidates
    i=0
    for i in range(10):
        sf_pd['negative'][i] = gl.distances.cosine(train_data_text[email+'_tfidf'][i],reproved['tfidf'][i])
        sf_pd['positive'][i] = gl.distances.cosine(train_data_text[email+'_tfidf'][i],approved['tfidf'][i])
    #convert back pandas to SFrame
    pd_sf = gl.SFrame(sf_pd)
    #update the distances for the candidate in the original SFrame
    train_data_text[email+'_positive'] = pd_sf['positive']
    train_data_text[email+'_negative'] = pd_sf['negative']

###Test Set

In [143]:
for email in candidates_test:
    #get the tf-idf for the candidate
    test_data_text[email+'_word_count'] = gl.text_analytics.count_words(test_data_text[email])
    tfidf = gl.text_analytics.tf_idf(test_data_text[email+'_word_count'])
    test_data_text[email+'_tfidf'] = tfidf['docs']
    #initialize the positive and negative column
    test_data_text[email+'_positive'] = test_data_text[email].apply(lambda x: 0.8000000)
    test_data_text[email+'_negative'] = test_data_text[email].apply(lambda x: 0.8000000)
    #prepare a SFrame to be converted to pandas
    sf = gl.SFrame({"positive":test_data_text[email+'_positive'],"negative":test_data_text[email+'_negative']})
    #convert SFrame to pandas because columns in SFrame are immutable, no access to rows
    sf_pd = sf.to_dataframe()
    #Calculate the distance from tf-idf candidate to approved and reproved candidates
    i=0
    for i in range(10):
        sf_pd['negative'][i] = gl.distances.cosine(test_data_text[email+'_tfidf'][i],reproved['tfidf'][i])
        sf_pd['positive'][i] = gl.distances.cosine(test_data_text[email+'_tfidf'][i],approved['tfidf'][i])
    #convert back pandas to SFrame
    pd_sf = gl.SFrame(sf_pd)
    #update the distances for the candidate in the original SFrame
    test_data_text[email+'_positive'] = pd_sf['positive']
    test_data_text[email+'_negative'] = pd_sf['negative']

##Save Processed Data

In [145]:
train_data_text.save(path+'train_data_processed.csv', format='csv')
cd_train.to_csv(path+'train_data_candidates.csv', index=False)
test_data_text.save(path+'test_data_processed.csv', format='csv')
cd_test.to_csv(path+'test_data_candidates.csv', index=False)